# PC 3 

Alumna: Ariadna Chávez Aliaga

### Pregunta 1

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

Importamos la data de carseats.csv

In [3]:
data = pd.read_csv('C:/Users/arica/Documents/GitHub/Qlab_Repositorio/Introducción a Machine Learning/PC3/carseats.csv')

data

,Unnamed: 0,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [5]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X = data.drop('Sales', axis=1)  # independientes
y = data['Sales']  # dependiente

# 70% entrenamiento, 30% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Observamos primeras filas
data.head(), X_train.shape, X_test.shape

(   Unnamed: 0  Sales  CompPrice  Income  Advertising  Population  Price  \
 0           0   9.50        138      73           11         276    120   
 1           1  11.22        111      48           16         260     83   
 2           2  10.06        113      35           10         269     80   
 3           3   7.40        117     100            4         466     97   
 4           4   4.15        141      64            3         340    128   
 
   ShelveLoc  Age  Education Urban   US  
 0       Bad   42         17   Yes  Yes  
 1      Good   65         10   Yes  Yes  
 2    Medium   59         12   Yes  Yes  
 3    Medium   55         14   Yes  Yes  
 4       Bad   38         13   Yes   No  ,
 (280, 11),
 (120, 11))

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificando las columnas categóricas
categorical_features = data.select_dtypes(include=['object']).columns

# Creando un transformador de columnas para la codificación de las variables categóricas
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Creando un árbol de regresión con el transformador de columnas
regressor = Pipeline(steps=[('preprocessor', column_transformer),
                            ('regressor', DecisionTreeRegressor(random_state=42))])

# Ajustando el modelo al conjunto de entrenamiento
regressor.fit(X_train, y_train)

# Realizando predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)

# Calculando el MSE para el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
mse



4.7374616666666665

El modelo de árbol de regresión ha sido ajustado al conjunto de entrenamiento. El valor del Mean Squared Error (MSE) para el conjunto de prueba es aproximadamente 4.737.

Ahora, pasemos al tercer paso: aplicar el método Bagging y calcular el MSE para el conjunto de prueba.

In [8]:
# Creando un modelo Bagging con árboles de decisión como base
bagging_regressor = Pipeline(steps=[('preprocessor', column_transformer),
                                    ('regressor', BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=42),
                                                                   random_state=42))])

# Ajustando el modelo Bagging al conjunto de entrenamiento
bagging_regressor.fit(X_train, y_train)

# Realizando predicciones en el conjunto de prueba
y_pred_bagging = bagging_regressor.predict(X_test)

# Calculando el MSE para el conjunto de prueba con Bagging
mse_bagging = mean_squared_error(y_test, y_pred_bagging)
mse_bagging



TypeError: BaggingRegressor.__init__() got an unexpected keyword argument 'base_estimator'